# Test code

### 1.1 Datasets preprocessing

In [ ]:
python full_preprocessing.py --mimic-path '/data/wolf6245/src/mm_study/data/a_raw/MIMIC/MIMIC-IV/' --zip-path '/data/wolf6245/src/mm_study/data/a_raw/MIMIC/MIMIC-IV-ECG/' --target-path '/data/wolf6245/src/ECG-MIMIC/data' --filter_file "/data/wolf6245/src/mm_study/data/f_modelling/03_model_input/data-2024-12-19-01-23-23/(3) Chronic ischaemic heart disease/y_fusion_label_not_gt.parquet"

## Filter

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
# Define data
df_filter = pd.read_parquet("/data/wolf6245/src/mm_study/data/f_modelling/03_model_input/data-2024-12-19-01-23-23/(3) Chronic ischaemic heart disease/y_fusion_label_not_gt.parquet")
df_folds = pd.read_pickle("/data/wolf6245/src/mm_study/data/f_modelling/03_model_input/data-2024-12-19-01-23-23/(3) Chronic ischaemic heart disease/train_test_vali_folds_fusion_label.pkl")
subject_ids_to_use = df_filter['subject_id'].unique()
hadm_ids_to_use = df_filter['hadm_id'].unique()
folder_processed_records = "/data/wolf6245/src/ECG-MIMIC/data/processed"

# Load data
records = pd.read_pickle("/data/wolf6245/src/ECG-MIMIC/data/records.pkl")
records_w_diag = pd.read_pickle("/data/wolf6245/src/ECG-MIMIC/data/records_w_diag.pkl")
records_w_diag_icd10 = pd.read_csv("/data/wolf6245/src/ECG-MIMIC/data/records_w_diag_icd10.csv")
std = np.load("/data/wolf6245/src/ECG-MIMIC/data/processed/std.npy")
mean = np.load("/data/wolf6245/src/ECG-MIMIC/data/processed/mean.npy")
lbl_itos = np.load("/data/wolf6245/src/ECG-MIMIC/data/processed/lbl_itos.npy")
df = pd.read_pickle("/data/wolf6245/src/ECG-MIMIC/data/processed/df.pkl")
memmap_meta = np.load("/data/wolf6245/src/ECG-MIMIC/data/memmap/memmap_meta.npz", allow_pickle=True)
df_memmap = pd.read_pickle("/data/wolf6245/src/ECG-MIMIC/data/memmap/df_memmap.pkl")
# memmap = np.load("/data/wolf6245/src/ECG-MIMIC/data/memmap/memmap.npy", allow_pickle=True)

# Filter to subject_ids
records_filtered = records[records['patient_id'].isin(subject_ids_to_use)].copy()
records_w_diag_filtered = records_w_diag[records_w_diag['subject_id'].isin(subject_ids_to_use)].copy()
records_w_diag_icd10_filtered = records_w_diag_icd10[records_w_diag_icd10['subject_id'].isin(subject_ids_to_use)].copy()
df_filtered = df[df['subject_id'].isin(subject_ids_to_use)].copy()
df_memmap_filtered = df_memmap[df_memmap['subject_id'].isin(subject_ids_to_use)].copy()
print(f"Filtered subject_ids: records from {len(records)} to {len(records_filtered)}")
print(f"Filtered subject_ids: records_w_diag from {len(records_w_diag)} to {len(records_w_diag_filtered)}")
print(f"Filtered subject_ids: records_w_diag_icd10 from {len(records_w_diag_icd10)} to {len(records_w_diag_icd10_filtered)}")
print(f"Filtered subject_ids: df from {len(df)} to {len(df_filtered)}")
print(f"Filtered subject_ids: df_memmap from {len(df_memmap)} to {len(df_memmap_filtered)}")

# Filter to hadm_ids
records_w_diag_filtered_len = len(records_w_diag_filtered)
records_w_diag_icd10_filtered_len = len(records_w_diag_icd10_filtered)
df_filtered_len = len(df_filtered)
records_filtered_len = len(records_filtered)
records_w_diag_filtered = records_w_diag_filtered.dropna(subset=['hosp_hadm_id'])
records_w_diag_filtered = records_w_diag_filtered[records_w_diag_filtered['hosp_hadm_id'].astype(int).isin(hadm_ids_to_use)].copy()
records_w_diag_icd10_filtered = records_w_diag_icd10_filtered.dropna(subset=['hosp_hadm_id'])
records_w_diag_icd10_filtered = records_w_diag_icd10_filtered[records_w_diag_icd10_filtered['hosp_hadm_id'].astype(int).isin(hadm_ids_to_use)].copy()
study_ids_to_use = records_w_diag_filtered['study_id'].unique()
df_filtered = df_filtered[df_filtered['study_id'].isin(study_ids_to_use)].copy()
records_filtered = records_filtered[records_filtered['study_id'].isin(study_ids_to_use)].copy()
df_memmap_filtered_len = len(df_memmap_filtered)
df_memmap_filtered = df_memmap_filtered[df_memmap_filtered['study_id'].isin(study_ids_to_use)].copy()
print(f"Filtered hadm_ids: records_w_diag from {records_w_diag_filtered_len} to {len(records_w_diag_filtered)}")
print(f"Filtered hadm_ids: records_w_diag_icd10 from {records_w_diag_icd10_filtered_len} to {len(records_w_diag_icd10_filtered)}")
print(f"Filtered study_ids: df from {df_filtered_len} to {len(df_filtered)}")
print(f"Filtered study_ids: records from {records_filtered_len} to {len(records_filtered)}")
print(f"Filtered study_ids: df_memmap from {df_memmap_filtered_len} to {len(df_memmap_filtered)}")

# Filter to only one row per subject_id
if True:
    records_filtered = records_filtered.sort_values(by=['patient_id', 'study_id']).drop_duplicates(subset=['patient_id'], keep='first')
    records_w_diag_filtered = records_w_diag_filtered.sort_values(by=['subject_id', 'study_id']).drop_duplicates(subset=['subject_id'], keep='first')
    records_w_diag_icd10_filtered = records_w_diag_icd10_filtered.sort_values(by=['subject_id', 'study_id']).drop_duplicates(subset=['subject_id'], keep='first')
    df_filtered = df_filtered.sort_values(by=['subject_id', 'study_id']).drop_duplicates(subset=['subject_id'], keep='first')
    df_memmap_filtered = df_memmap_filtered.sort_values(by=['subject_id', 'study_id']).drop_duplicates(subset=['subject_id'], keep='first')
    print(f"Filtered records_filtered from {len(records_filtered)} to {len(records_filtered)}")
    print(f"Filtered records_w_diag_filtered from {len(records_w_diag_filtered)} to {len(records_w_diag_filtered)}")
    print(f"Filtered records_w_diag_icd10_filtered from {len(records_w_diag_icd10_filtered)} to {len(records_w_diag_icd10_filtered)}")
    print(f"Filtered df_filtered from {len(df_filtered)} to {len(df_filtered)}")
    print(f"Filtered df_memmap_filtered from {len(df_memmap_filtered)} to {len(df_memmap_filtered)}")

# Add folds
df_filtered['fold'] = np.nan
df_memmap_filtered['fold'] = np.nan
for i, fold in enumerate(df_folds):
    test_ids = fold[1]
    df_filtered.loc[df_filtered["subject_id"].isin(test_ids), "fold"] = i
    df_memmap_filtered.loc[df_memmap_filtered["subject_id"].isin(test_ids), "fold"] = i

# Save filtered data back
if True:
    records_filtered.to_pickle("/data/wolf6245/src/ECG-MIMIC/data/records.pkl")
    records_w_diag_filtered.to_pickle("/data/wolf6245/src/ECG-MIMIC/data/records_w_diag.pkl")
    records_w_diag_icd10_filtered.to_csv("/data/wolf6245/src/ECG-MIMIC/data/records_w_diag_icd10.csv", index=False)
    df_filtered.to_pickle("/data/wolf6245/src/ECG-MIMIC/data/processed/df.pkl")
    df_memmap_filtered.to_pickle("/data/wolf6245/src/ECG-MIMIC/data/memmap/df_memmap.pkl")

# Delete files not in df_filtered.data
if False:
    files_to_keep = df_filtered['data'].unique()
    all_files = os.listdir(folder_processed_records)
    all_files = [f for f in all_files if f.startswith('p')]
    files_to_delete = [f for f in all_files if f not in files_to_keep]
    # for f in files_to_delete:
    #     os.remove(os.path.join(folder_processed_records, f))
    print(f"Deleted {len(files_to_delete)} files from {len(all_files)} files in {folder_processed_records}")

Filtered subject_ids: records from 55720 to 55720
Filtered subject_ids: records_w_diag from 55720 to 55720
Filtered subject_ids: records_w_diag_icd10 from 55720 to 55720
Filtered subject_ids: df from 55720 to 55720
Filtered subject_ids: df_memmap from 55720 to 55720
Filtered hadm_ids: records_w_diag from 55720 to 17200
Filtered hadm_ids: records_w_diag_icd10 from 55720 to 17200
Filtered study_ids: df from 55720 to 17200
Filtered study_ids: records from 55720 to 17200
Filtered study_ids: df_memmap from 55720 to 17200
Filtered records_filtered from 5278 to 5278
Filtered records_w_diag_filtered from 5278 to 5278
Filtered records_w_diag_icd10_filtered from 5278 to 5278
Filtered df_filtered from 5278 to 5278
Filtered df_memmap_filtered from 5278 to 5278


In [7]:
from pykeops.config import set_build_options, clean_keops

set_build_options(cuda_type="nvcc")
clean_keops()

# Then import and run your model code


ImportError: cannot import name 'set_build_options' from 'pykeops.config' (/data/wolf6245/envs/miniconda3/envs/ecg/lib/python3.11/site-packages/pykeops/config.py)

### 1.2 Models training

In [ ]:
python main_ecg.py --data "/data/wolf6245/src/ECG-MIMIC/data" --input-size 250 --finetune-dataset mimic_ed_all_edfirst_all_2000_5A --architecture s4 --precision 32 --s4-n 8 --s4-h 512 --batch-size 32 --epochs 20 --export-predictions-path "T(ED2ALL)-E(ED2ALL)/" > "T(ED2ALL)-E(ED2ALL).log"